In [37]:
import random
import numpy as np


def score(context, response):
    return random.random()


def best_response(context, candidates):
    index = np.argmax([score(context, response) for response in candidates])
    return candidates[index]


def parse_dialogs(filename):
    dialogs = []
    with open(filename, 'r') as f:
        dialog = []
        for line in f:
            if line.strip() == '':
                dialogs.append(dialog)
                dialog = []
            else:
                user_utt, bot_utt = line.strip().split('\t')
                utt_num = user_utt.split(' ')[0]
                user_utt = ' '.join(user_utt.split(' ')[1:])
                dialog.append((utt_num, user_utt, bot_utt))
    return dialogs


def parse_candidates(filename):
    with open(filename, 'r') as f:
        return [' '.join(line.strip().split(' ')[1:]) for line in f]            

    
def responses_accuracy(dialogs, candidates):
    correct = 0
    count = 0
    for dialog in dialogs:
        for _, user_utt, bot_utt in dialog:
            count += 1
            context = user_utt
            response = best_response(context, candidates)
            if response == bot_utt:
                correct += 1
    return correct / count, correct, count

In [38]:
dev_set_task1_dialogs = parse_dialogs('dataset/dialog-bAbI-tasks/dialog-babi-task1-API-calls-dev.txt')
candidates = parse_candidates('dataset/dialog-bAbI-tasks/dialog-babi-candidates.txt')

In [39]:
responses_accuracy(dev_set_task1_dialogs, candidates)

(0.00033250207813798836, 2, 6015)